### Comparisson Notebook

 This notebook is used to generate comparisson plots between our implemntation, SP's implementation, and stylegan2 to show the differences in controllability between the three approaches and show each approach's capability.

In [1]:
import os
import argparse
import shutil
import numpy as np
import imageio
from itertools import chain
import torch
import sys
import matplotlib.pyplot as plt
sys.path.insert(0, "../../SemanticStyleGAN")
from models import make_model
from visualize.utils import generate, cubic_spline_interpolate
from utils.control import Control
from numpy import array
from numpy import mean
from numpy import cov
from numpy.linalg import eig
sys.path.insert(0,"/no_backups/g013/other_GANs/stylegan2-pytorch")
from model import Generator as StyleGANGenerator


OSError: CUDA_HOME environment variable is not set. Please set it to your CUDA install root.

#### Manipulate images using Semantic Palette
Manipulation itself is done on the side of semantic palette's code of visualization ( interpolation of semantic condition) here we mainly only change the condition $t$

In [4]:
sem0_18 = torch.load(f"./data_for_comparison/sem_0_18.pt")
sem1_18 = torch.load(f"./data_for_comparison/sem_1_18.pt")
sem0_20 = torch.load(f"./data_for_comparison/sem_0_20.pt")
sem1_20 = torch.load(f"./data_for_comparison/sem_1_20.pt")

In [ ]:
#Generating a new sem1 by modifying sem0 which has a high dist of road and vegitation to have 
#instead a higher dist of cars

#Editing Image 18
car_index=26
vegi_index=21
road_index=7
new_sem = sem0_18.detach().clone()
print(f"distribution of road in sem0 :{sem0_18[0][road_index]*100}  vegetation {sem0_18[0][vegi_index]*100} car :{sem0_18[0][car_index]*100}")
new_sem[0][road_index]-=0.15
new_sem[0][vegi_index]-=0.05
new_sem[0][car_index]+=0.20
print(f"distribution of road in new_sem :{new_sem[0][road_index]*100}  vegetation {new_sem[0][vegi_index]*100} car :{new_sem[0][car_index]*100}")

torch.save(new_sem,f"./data_for_comparison/sem_1_18_edited.pt")
new_sem.sum()

distribution of road in sem0 :39.28250503540039  vegetation 11.405961990356445 car :3.828788995742798
distribution of road in new_sem :24.282506942749023  vegetation 6.405961513519287 car :23.82878875732422


tensor(1.)

In [31]:
#Editing Image 20
car_index=26
vegi_index=21
road_index=7
new_sem = sem0_20.detach().clone()
print(f"distribution of road in sem0 :{sem0_20[0][road_index]*100}  vegetation {sem0_20[0][vegi_index]*100} car :{sem0_20[0][car_index]*100}")
new_sem[0][road_index]-=0.15
new_sem[0][vegi_index]-=0.05
new_sem[0][car_index]+=0.20
print(f"distribution of road in new_sem :{new_sem[0][road_index]*100}  vegetation {new_sem[0][vegi_index]*100} car :{new_sem[0][car_index]*100}")

torch.save(new_sem,f"./data_for_comparison/sem_1_20_edited.pt")
new_sem.sum()

distribution of road in sem0 :32.07407760620117  vegetation 6.262269020080566 car :21.392187118530273
distribution of road in new_sem :17.074077606201172  vegetation 1.2622687816619873 car :41.392189025878906


tensor(1.0000)

In [17]:
#

print(sem1_18[0][26])
print(sem1_18[0][7])

tensor(0.0149)
tensor(0.3883)


### Another comparisson:

In [8]:
sem0_4 = torch.load(f"/no_backups/g013/other_GANs/SemanticPalette/our_analysis/version_2/picked_image/sem_0_4.pt")
sem1_4 = torch.load(f"/no_backups/g013/other_GANs/SemanticPalette/our_analysis/version_2/picked_image/sem_1_4.pt")
sem0_2 = torch.load(f"/no_backups/g013/other_GANs/SemanticPalette/our_analysis/version_3/picked_samples/sem_0_2.pt")
sem1_2 = torch.load(f"/no_backups/g013/other_GANs/SemanticPalette/our_analysis/version_3/picked_samples/sem_1_2.pt")


In [17]:
#Generating a new sem1 by modifying sem0 which has a high dist of road and vegitation to have 
#instead a higher dist of cars

#Editing Image 4
car_index=26
vegi_index=21
road_index=7




#Increase Road
new_sem = sem1_2.detach().clone()
print(f"distribution of road in sem0 :{new_sem[0][road_index]*100}  vegetation {new_sem[0][vegi_index]*100} car :{new_sem[0][car_index]*100}")
new_sem[0][road_index]+=0.13
new_sem[0][vegi_index]-=0.08
new_sem[0][car_index]-=0.05
print(new_sem.sum())
print(f"distribution of road in sem0 :{new_sem[0][road_index]*100}  vegetation {new_sem[0][vegi_index]*100} car :{new_sem[0][car_index]*100}")
torch.save(new_sem,"/no_backups/g013/other_GANs/SemanticPalette/our_analysis/version_3/picked_samples/sem_0_2_more_road.pt")

#Decrease Road
new_sem = sem1_2.detach().clone()
print(f"distribution of road in sem0 :{new_sem[0][road_index]*100}  vegetation {new_sem[0][vegi_index]*100} car :{new_sem[0][car_index]*100}")
new_sem[0][road_index]-=0.20
new_sem[0][vegi_index]+=0.15
new_sem[0][car_index]+=0.05
print(new_sem.sum())
print(f"distribution of road in sem0 :{new_sem[0][road_index]*100}  vegetation {new_sem[0][vegi_index]*100} car :{new_sem[0][car_index]*100}")
torch.save(new_sem,"/no_backups/g013/other_GANs/SemanticPalette/our_analysis/version_3/picked_samples/sem_0_2_less_road.pt")

#Increase Car
new_sem = sem1_2.detach().clone()
print(f"distribution of road in sem0 :{new_sem[0][road_index]*100}  vegetation {new_sem[0][vegi_index]*100} car :{new_sem[0][car_index]*100}")
new_sem[0][road_index]-=0.15
new_sem[0][vegi_index]-=0.05
new_sem[0][car_index]+=0.20
print(new_sem.sum())
print(f"distribution of road in sem0 :{new_sem[0][road_index]*100}  vegetation {new_sem[0][vegi_index]*100} car :{new_sem[0][car_index]*100}")
torch.save(new_sem,"/no_backups/g013/other_GANs/SemanticPalette/our_analysis/version_3/picked_samples/sem_0_2_more_car.pt")

#Decrease Car

new_sem = sem1_2.detach().clone()
print(f"distribution of road in sem0 :{new_sem[0][road_index]*100}  vegetation {new_sem[0][vegi_index]*100} car :{new_sem[0][car_index]*100}")
new_sem[0][road_index]+=0.045
new_sem[0][vegi_index]+=0.045
new_sem[0][car_index]-=0.09
print(new_sem.sum())
print(f"distribution of road in sem0 :{new_sem[0][road_index]*100}  vegetation {new_sem[0][vegi_index]*100} car :{new_sem[0][car_index]*100}")
torch.save(new_sem,"/no_backups/g013/other_GANs/SemanticPalette/our_analysis/version_3/picked_samples/sem_0_2_less_car.pt")


#Increase vegi
new_sem = sem1_2.detach().clone()
print(f"distribution of road in sem0 :{new_sem[0][road_index]*100}  vegetation {new_sem[0][vegi_index]*100} car :{new_sem[0][car_index]*100}")
new_sem[0][road_index]-=0.17
new_sem[0][vegi_index]+=0.20
new_sem[0][car_index]-=0.03
print(new_sem.sum())
print(f"distribution of road in sem0 :{new_sem[0][road_index]*100}  vegetation {new_sem[0][vegi_index]*100} car :{new_sem[0][car_index]*100}")
torch.save(new_sem,"/no_backups/g013/other_GANs/SemanticPalette/our_analysis/version_3/picked_samples/sem_0_2_more_vegi.pt")

#Decrease Vegi
new_sem = sem1_2.detach().clone()
print(f"distribution of road in sem0 :{new_sem[0][road_index]*100}  vegetation {new_sem[0][vegi_index]*100} car :{new_sem[0][car_index]*100}")
new_sem[0][road_index]+=0.10
new_sem[0][vegi_index]-=0.078
new_sem[0][car_index]+=0.05
print(new_sem.sum())
print(f"distribution of road in sem0 :{new_sem[0][road_index]*100}  vegetation {new_sem[0][vegi_index]*100} car :{new_sem[0][car_index]*100}")
torch.save(new_sem,"/no_backups/g013/other_GANs/SemanticPalette/our_analysis/version_3/picked_samples/sem_0_2_less_vegi.pt")

#torch.save(new_sem,f"./data_for_comparison/sem_1_18_edited.pt")
new_sem.sum()

distribution of road in sem0 :35.59489822387695  vegetation 8.172882080078125 car :10.403952598571777
tensor(1.)
distribution of road in sem0 :48.59489822387695  vegetation 0.17288252711296082 car :5.403952598571777
distribution of road in sem0 :35.59489822387695  vegetation 8.172882080078125 car :10.403952598571777
tensor(1.0000)
distribution of road in sem0 :15.594898223876953  vegetation 23.172882080078125 car :15.403953552246094
distribution of road in sem0 :35.59489822387695  vegetation 8.172882080078125 car :10.403952598571777
tensor(1.)
distribution of road in sem0 :20.594898223876953  vegetation 3.172882318496704 car :30.403953552246094
distribution of road in sem0 :35.59489822387695  vegetation 8.172882080078125 car :10.403952598571777
tensor(1.0000)
distribution of road in sem0 :40.09490203857422  vegetation 12.672883033752441 car :1.4039523601531982
distribution of road in sem0 :35.59489822387695  vegetation 8.172882080078125 car :10.403952598571777
tensor(1.)
distribution o

tensor(1.0720)

#### Manipulate images using SSG

Editing the 2 images using 2 directions basicaly. one for increasing all of the side cars in the image
and the other is for increasing the cars on the left for example.

In [5]:
ckpt= "/no_backups/g013/checkpoints/SSG_v3.13/ckpt/140000.pt"
device="cpu"
control = Control(ckpt,device)

Loading model ...
Initializing model with arguments:
{'aug': False,
 'base_layers': 2,
 'batch': 4,
 'channel_multiplier': 2,
 'checkpoint_dir': '/no_backups/g013/checkpoints/SSG_v3.13',
 'ckpt': None,
 'coarse_channel': 512,
 'coarse_size': 64,
 'd_reg_every': 16,
 'dataset': '/no_backups/g013/data/lmdb_datasets/lmdb_v3.3',
 'depth_layers': 6,
 'detach_texture': False,
 'distributed': True,
 'g_reg_every': 4,
 'inception': '/no_backups/g013/data/inception_models/inception_v3.3.pkl',
 'iter': 600001,
 'lambda_mask': 100.0,
 'latent': 512,
 'local_channel': 64,
 'local_layers': 10,
 'local_rank': 0,
 'lr': 0.002,
 'min_feat_size': 16,
 'mixing': 0.3,
 'n_gpu': 4,
 'n_mlp': 8,
 'n_sample': 16,
 'num_workers': 8,
 'path_batch_shrink': 2,
 'path_regularize': 0.5,
 'r1_img': 10,
 'r1_seg': 1000,
 'residual_refine': True,
 'save_every': 5000,
 'seg_dim': 16,
 'size': 256,
 'start_iter': 0,
 'transparent_dims': (10, 12),
 'viz_every': 2000}


In [6]:
device = "cpu"
latent_img_18= "./data/comparison_data/img_0_0_18.npy"
latent_img_20= "./data/comparison_data/img_0_0_20.npy"

styles_18 = torch.tensor(np.load(latent_img_18), device=device)
styles_20 = torch.tensor(np.load(latent_img_20), device=device)



In [10]:
## Generating sequence for 2 images Using PCA direction in W Space 
V = torch.load("data/principal_components/principal_components_2.pt")

## Effects of continous changing of 1 component
apply_to_all_layers=False
latent_index=28
class_index=13 #Car
latent_name="car_shape"
component=9

# chain_1 = range(0,0,1) # Set aside for now 
# chain_2 =  np.arange(10,-10,-0.1)
# mult_range = chain(chain_1,chain_2)

for component in [3]:
    print(f"Processing component {component}")
    for i,style_chosen in enumerate([styles_18]):
        print(f"Styles{i+1}")       
        images=[]
        segs=[]
        #for multiplier in  np.linspace(30,-40,70):
        for multiplier in  np.linspace(50,-60,70):
            #print(f"Analyzing COMPONENT {component} with multiplier {multiplier}")
            image,seg=control.edit_image_principal_component(latent_index,class_index,multiplier,style_chosen,V[component],whole_image=apply_to_all_layers,plot=False,get_image=True)
            images.append(image[0])
            segs.append(seg[0])
        # images = images + images[::-1]
        # segs= segs + segs[::-1]
        control.images_to_video(images,segs,f"./data/comparison_data/output/editing_principal_component_{component}_for_car{i+1}.mp4")


Processing component 3
Styles1
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in


###### Editing using Sefa based directions.

In [7]:
def convert_from_w_to_w_extended(model,w_vectors,local_generator,layer):
    '''
    A function that converts given w_vectors of size Nx512 to Nx64 given a specific demodulation in the model.
    '''
    modulation=control.model.__getattr__("local_nets")[local_generator].__getattr__("linears")[layer].__getattr__("conv").modulation
    return modulation(w_vectors)
    
def prepare_w_extended(model,style,w_extended,class_index,layers):
    assert len(style.shape)==1
    for layer_index in layers:
         w_extended[class_index][layer_index]=convert_from_w_to_w_extended(model,style,class_index,layer_index).detach()
    return w_extended

def calculate_sefa(model,local_nets,layers):
    '''
    Function to get certain weights out of the SSG Model.
    local_nets between 0 and 15
    layers between 0 and 9
    '''
    all_local_nets=model.__getattr__("local_nets")
    weights = []
    if not type(layers)==list:
        layers=[layers]
    for l_net in local_nets:
        for layer in layers:
            weight_temp=all_local_nets[l_net].__getattr__("linears")[layer].__getattr__("conv").weight.squeeze(0)
            weight_temp = weight_temp.flip(2, 3).permute(1, 0, 2, 3).flatten(1)
            weights.append(weight_temp.cpu().detach().numpy())
    weight = np.concatenate(weights, axis=1).astype(np.float32)
    weight = weight / np.linalg.norm(weight, axis=0, keepdims=True)
    eigen_values, eigen_vectors = np.linalg.eig(weight.dot(weight.T))

    return eigen_vectors.T,eigen_values

In [8]:
## Effects of continous changing of 1 component

apply_to_all_layers=False
latent_index=4


component=9
coarse_layer=1
w_extended=torch.zeros(16,10,64)
shape_layers=[2,3,4,5]
texture_layers=[6,7,8,9]
#Training list of format :
'''
[ [Class_index,LocalGeneratorLayer, [V_components]] , ...another training param..]
'''

                
training_list = [[13,[3],[1]],
                [13,[4],[0,3]],]

styles_list= [styles_20,styles_18]
saved_pcas=[]
#V=calculate_pca_from_s_space(control.model,class_index,layer_index)
for training_instance in training_list:
    class_index = training_instance[0]
    layer_index= training_instance[1]
    components = training_instance[2]
    print(f"Processing class_index : {class_index}  layer index: {layer_index} with components {components}")
    V,_=calculate_sefa(control.model,[class_index],layer_index)
    #V=calculate_pca_from_s_space(control.model,class_index,layer_index[0])
    #saved_pcas.append(V)
    #np.save(f"./thesis_related_results/saved_numpy/layer_{layer_index[0]}_class_index_{class_index}.npy",V)
    #V = np.load(f"data/pca_new_analysis/pca_for_layer_{}.npy")
    for component in components:
        print(f"Processing component {component}")
        for i,style_chosen in enumerate(styles_list):
            print(f"Styles{i+1}")    
            images=[]
            segs=[]
            for multiplier in  np.linspace(-60,60,45):
                w_extended_copy = w_extended.clone().detach()
                w_extended_copy=prepare_w_extended(control.model,style_chosen[0][0],w_extended_copy,class_index,layer_index)
                w_extended_copy[class_index][layer_index[0]]+=(multiplier*V[component])
                #print(f"Analyzing COMPONENT {component} with multiplier {multiplier}")
                image,seg= control.edit_image_inject_modulation(class_index,style_chosen,w_extended_copy,plot=False,get_image=True)
                images.append(image[0])
                segs.append(seg[0])

            control.images_to_video(images,segs,f"./data/comparison_data/output/SEFA_S_{component}_component_for_image_{i+1}__class_{class_index}_layer_{layer_index}.mp4")

Processing class_index : 13  layer index: [3] with components [1]
Processing component 1
Styles1
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
Styles2
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
Processing class_index : 13  layer index: [4] with components [0, 3]
Processing component 0
Styles1
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
Styles2
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
Processing component 3
Styles1
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
Styles2
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
i

### StyleGAN2 Comparisson

###### First we will explore some of StyleGAN2's latent space using GANSPACE, Then we will invert an image that we will explore both on the side of stylegan2 and SSG.

#### Exploring StyleGAN2 LatentSpace

In [16]:
def calculate_pca_2(samples,selected=10):
    samples_cop = samples.cpu().detach().numpy()
    M = mean(samples_cop)
    C = samples_cop-M
    V_2=cov(C.T)
    values, vectors = eig(V_2)
    return values[:selected],vectors[:selected]

In [20]:
#Load StyleGAN2 Model

ckpt="/no_backups/g013/checkpoints/StyleGAN2_CityScapes/250000__16.pt"
latent = 512
n_mlp = 8
size=256
device="cpu"
g_ema = StyleGANGenerator(
    size,latent, n_mlp, channel_multiplier=2
).to(device)
checkpoint = torch.load(ckpt,map_location=torch.device('cpu'))
g_ema.load_state_dict(checkpoint["g_ema"],strict=False)

<All keys matched successfully>

In [18]:
#Sample N style vectors to calculate PCA
sample_z = torch.randn(50000, latent, device=device)
sample_w = g_ema.get_latent(sample_z)


#Calculate PCA and retrieve eigenvectors.
# res = calculate_pca_2(sample_w,selected=513)
# vectors = res[1]
# vectors = torch.tensor(vectors)
# torch.save(vectors,"data/comparison_data/principal_components_style_gan.pt")

In [101]:
### Explore StyleGAN2 directions:
# st1 = torch.load("data/stylegan2_samples/bielefeld_000000_005068_leftImg8bit.pt",map_location=torch.device('cpu')) 
# st2 = torch.load("data/stylegan2_samples/bielefeld_000000_033979_leftImg8bit.pt",map_location=torch.device('cpu')) 
# style_1=st1[list(st1.keys())[0]]["latent"].unsqueeze(0)
# style_2=st2[list(st2.keys())[0]]["latent"]

# print(style_1.shape)
# print(style_2.shape)


In [27]:
def images_to_video(images, save_dir,fps=10):
    frames = images
    imageio.mimwrite(f"{save_dir}", frames, fps=fps)


def edit_image_principal_component(
    g_ema,
    mean_latent,
    change_factor,
    styles,
    principal_component,
    get_image=True,
    plot=False,
):
    styles_copy = styles.clone().detach()
    styles_copy = styles_copy + (principal_component.float() * change_factor)
    image, _ = g_ema(
    styles_copy, truncation=1, truncation_latent=mean_latent,input_is_latent=True,
        )
    image=image[0].permute(1,2,0).detach().numpy()
    image_new = ((image - image.min()) * (1/(image.max() - image.min()) * 255)).astype('uint8')

    if plot:
        plt.imshow(image_new)
        plt.show()
    if get_image:
        return image

In [61]:
# ##Sampling a new image
# sample_z = torch.randn(1,latent,device=device)
# sample_w = g_ema.get_latent(sample_z).unsqueeze(0)
# mean_latent = g_ema.mean_latent(4096)
# torch.save(sample_w,"./data/stylegan2_results/input_image.pt")

In [60]:


apply_to_all_layers=False
latent_index=28
class_index=13
latent_name="car_shape"
component=9

V= torch.load("data/comparison_data/principal_components_style_gan.pt",map_location=torch.device('cpu')) 

for component in [0]:
    print(f"Processing component {component}")
    for i,style_chosen in enumerate([sample_w]):
        print(f"Styles{i+1}")       
        images=[]
        segs=[]
        for multiplier in  np.linspace(0,1,2):
            #print(f"Analyzing COMPONENT {component} with multiplier {multiplier}")
            image=edit_image_principal_component(g_ema,mean_latent,multiplier,sample_w,V[component],get_image=True,plot=False)
            images.append(image)
        # images = images + images[::-1]
        # segs= segs + segs[::-1]
    images_to_video(images,f"./data/stylegan2_results/editing_principal_component_{component}.mp4",fps=20)


Processing component 0
Styles1
torch.Size([1, 1, 512])
torch.Size([1, 1, 512])


Lossy conversion from float32 to uint8. Range [-0.9711652398109436, 0.8922786116600037]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [-0.9634853601455688, 0.8119815587997437]. Convert image to uint8 prior to saving to suppress this warning.
